---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1184]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
test = df[:]
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [1116]:
print(df.shape)
print(test.shape)

(500,)
(500,)


### Found 
- nums_only = 04/20/2009; 04/20/09; 4/20/09; 4/3/09
- alpha_present = 'Mar 20, 2009', 'March 20, 2009','20 Mar 2009','20 March 2009','Feb 2009','Sep 2009','Oct 2010'
'''

In [796]:
example = '''
Mar-20-2009
Mar. 20, 2009
Mar 20 2009
20 Mar. 2009
20 March, 2009
Mar 20th, 2009
Mar 21st, 2009
Mar 22nd, 2009

6/2008
12/2009
2009
2010'''

In [797]:
def count_empties(this_df):
    counter = 0
    for each in this_df:
        if (each == []) or (type(each) == str):
            counter+=1
    print('Dates Missing {}'.format(counter))
    found = this_df.shape[0] - counter
    print('Dates Found {}'.format(found))

In [798]:
# m(m)/d(d)/yy(yy), m(m)/d(d)/yy
# numbers only: 125
nums_only = r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}'
nums_only_df = test.str.findall(nums_only)

In [799]:
re.findall(nums_only,example)

[]

In [800]:
count_empties(nums_only_df)

Dates Missing 375
Dates Found 125


In [804]:
# inspect index 72
nums_only_df[101:126]

101      [10/05/97]
102    [07/18/2002]
103       [9/22/82]
104       [2/24/74]
105       [2/03/78]
106     [2/11/2006]
107       [8/22/83]
108       [5/04/74]
109     [7/20/2011]
110       [6/17/95]
111       [6/10/72]
112      [10/16/82]
113      [12/15/92]
114       [12/8/97]
115       [4/05/89]
116      [12/04/87]
117       [6/20/77]
118     [4/27/2006]
119      [07/17/92]
120      [12/22/98]
121      [10/02/96]
122      [11/05/90]
123       [5/04/77]
124       [2/27/96]
125              []
dtype: object

In [805]:
# to grab dates with Month at the beginning
#alpha_present = r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{2}, )?\d{4}'
#alpha_present = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
alpha_v2 = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Sept|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
alpha_present_df = df.str.findall(alpha_v2)
count_empties(alpha_present_df)

Dates Missing 282
Dates Found 218


In [806]:
alpha_present_df.loc[178]

['15 Jun 1985']

In [204]:
c = '''
Mar 20th, 2009
Mar-20-2009
Mar 20 2009
Mar 20th, 2009
Mar 21st, 2009
Mar 22nd, 2009
'''

In [807]:
alpha_present = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
gottem = re.findall(alpha_present, b)
print(len(gottem))
gottem


11


['20 Mar. 2009',
 '20 March, 2009',
 'Mar 20th, 2009',
 'Mar-20-2009',
 'Mar. 20, 2009',
 'Mar 20 2009',
 '20 Mar. 2009',
 '20 March, 2009',
 'Mar 20th, 2009',
 'Mar 21st, 2009',
 'Mar 22nd, 2009']

In [808]:
count_empties(alpha_present_df)

Dates Missing 282
Dates Found 218


In [809]:
218 + 125

343

In [810]:
def merge_found(x,y):
    if x == []:
        return y
    else:
        return x 

In [811]:
def insert_missing(got_it,missing):
    if len(got_it) == 0:
        return missing
        return got_it

In [812]:
found = alpha_present_df.combine(nums_only_df,merge_found)

In [318]:
single_dates = combined_df[missing]

In [320]:
#alpha_present = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
nums_dates = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
#single_dates

(157,)

In [461]:
combined_df

0                                             [03/25/93]
1                                              [6/18/85]
2                                               [7/8/71]
3                                              [9/27/75]
4                                               [2/6/96]
5                                              [7/06/79]
6                                              [5/18/78]
7                                             [10/24/89]
8                                               [3/7/86]
9                                              [4/10/71]
10                                             [5/11/85]
11                                             [4/09/75]
12                                             [8/01/98]
13                                             [1/26/72]
14                                           [5/24/1990]
15                                           [1/25/2011]
16                                             [4/12/82]
17                             

In [562]:
nums_dates = r'\d{,2}[/]+\d{2,4}'
nums_dates_df = combined_df.str.findall(nums_dates)
cnt = 0
nums_dates_good_index = []
for entry,idx in zip(nums_dates_df,nums_dates_df.index.tolist()):
    if type(entry) == list:
        if len(entry) > 0:
            print(entry)
            cnt += 1
            nums_dates_good_index.append(idx)
print(cnt)

['6/1998']
['6/2005']
['10/1973']
['9/2005']
['03/1980']
['12/2005']
['5/1987']
['5/2004']
['8/1974']
['3/1986']
['10/1997']
['3/1993']
['3/1981']
['9/2003']
['10/1993']
['1/1983']
['7/1994']
['12/2008']
['10/1980']
['8/2003']
['12/1975']
['11/2010']
['7/1997']
['7/2014']
['9/2001']
['8/1986']
['1/1978']
['9/1975']
['1/2009']
['5/1995']
['8/1989']
['11/2000']
['06/1973']
['2/1999']
['6/2001']
['12/1978']
['8/2009']
['7/1973']
['1/2014']
['09/1975']
['12/2012']
['5/1999']
['7/1989']
['7/2009']
['11/1998']
['3/1995']
['2/2009']
['08/1988']
['4/2007']
['5/2000']
['4/2012']
['10/2001']
['2/1977']
['8/2008']
['2/1983']
['5/1979']
['1/1992']
['11/2008']
['12/2014']
['4/1974']
['10/1981']
['10/1986']
['03/1973']
['12/1994']
['8/1999']
['10/2010']
['10/1994']
['01/2007']
['8/2010']
['03/1990']
['11/2016']
['4/2004']
['2/1973']
['7/1987']
['8/2000']
['8/1975']
['5/1977']
['3/2000']
['9/1984']
['4/1973']
['12/1986']
['4/1979']
['7/2004']
['11/1984']
['5/2016']
['12/2007']
['11/1982']
['06/1981']

In [563]:
nums_dates_good_index

[343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454]

In [566]:
# add the newly found dates of the m(m)/(yy)yy format
combined_df[nums_dates_good_index] = nums_dates_df[nums_dates_good_index]

In [571]:
count_empties(combined_df)

Dates Missing 45
Dates Found 455


In [535]:
def not_null(a,b):
    if a.isnull():
        return b
    else:
        return a

In [524]:
def merge_missing(a,b):
    if type(a) == list:
        return a
    else:
        return b

In [544]:
count_empties(found)

Dates Missing 157
Dates Found 343


In [546]:
combined_df.iloc[343]

'6/1998 Primary Care Doctor:\n'

In [302]:
missing = []
found_idx = found.nonzero()
for each in found.index.tolist():
    if each not in found_idx[0]:
        missing.append(each)

In [304]:
# if the entry is a list, we have the answer.  i
# if the entry is a str it is missing
combined_df = found.combine(df,insert_missing)

In [454]:
count = 0
for each in combined_df:
    if type(each) != list:
        count += 1
count

157

In [455]:
df[missing].iloc[0]

'6/1998 Primary Care Doctor:\n'

In [263]:
alpha_v2 = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'

In [276]:
re.findall(alpha_v2,combined_df.loc[178])

['15 Jun 1985']

In [264]:
re.findall(alpha_v2,df[missing].iloc[0])

['15 Jun 1985']

In [89]:
short_nums_example = '''
6/2008
12/2009
2009
2010
'''

In [137]:
short_nums_only = r'(\d{,2}[/-]?\d{2,4})'
re.findall(short_nums_only,short_nums_example)

['6/2008', '12/2009', '2009', '2010']

In [139]:
def still_missing(x):
    if type(x) == list:
        return ""
    else:
        return x

In [141]:
temp_series = found2.apply(still_missing)

In [530]:
short_nums_df = temp_series.str.findall(short_nums_only)
found2[short_nums_df.nonzero()[0]]

178    15 Jun 1985 @ 11 AMCommunication with referrin...
198                                 [ October. 11, 2013]
199                                       [ Jan 24 1986]
204                                      [Mar. 10, 1976]
206                                   [ October 23 1990]
207                                    [ August 12 2004]
214                                     [ Sep. 10, 1974]
219                                       [ Dec 14 1975]
223                                   [ October 14 1974]
227                                [September. 15, 2011]
233                                        [ July, 1990]
234    . Patient presents with GAD flareup in Jul 200...
240                                         [ May, 2011]
244                                     [ January, 2013]
262    hHelen Ernst, M.D. last appt Jun 2002, no othe...
270                                          [May, 2006]
273                                    [ February, 2010]
306                            

In [488]:
print(test.iloc[499])

. In 1980, patient was living in Naples and described being upset by the amount of noise generated by the neighbor living under his apartment.  He noted the woman was a heroin user and when she refused to quiet down, he walked into her apartment and "trashed it".  He was arrested for breaking and entering and destruction of property and was placed on probation.



In [98]:
count_empties(short_nums_df)

Dates Missing 0
Dates Found 500


In [573]:
count_empties(combined_df)

Dates Missing 45
Dates Found 455


In [588]:
[each for each in combined_df if type(each) == str]

['sHemmorage caused by probe in 1984 Medical History:\n',
 'sHas been at MYH since his treaters in NE retired in 2000. Was seen in NE for 20 years. Previouysly seen by Drs. Brandy Ivery and Quinn Prater. Prior medication trials (including efficacy, reasons discontinued):\n',
 'Pt joined Army reserves in 2001 and has 3 years left in this commitment.-Mental Status Exam Was the exam performed? (If not, indicate reason): Yes\n',
 'one sister from whom he is estranged due to her opiate dependence, legal conflict over mother\'s house following father\'s death in 1982, "I dont\' want to see family ever again"Past verbal, emotional, physical, sexual abuse: Patient Declined\n',
 'sSince 1998. Prior medication trials (including efficacy, reasons discontinued):\n',
 '1 Ex-smoker : quit 2012\n',
 '. Age 16, 1991, frontal impact. out for two weeks from sports.\n',
 'sLexapro (1988-now): Good response (anxiety)\n',
 's      25 year old engaged to be married Optician of Irish descent. She has a histo

In [594]:
# regex for year only
only_year = r'[a-zA-Z]?(\d{2,4})'
only_year_df = combined_df.str.findall(only_year)

In [598]:
def get_good_index(temp_df):
    cnt = 0
    good_index = []
    for entry,idx in zip(temp_df,temp_df.index.tolist()):
        if type(entry) == list:
            if len(entry) > 0:
                print(entry)
                cnt += 1
                good_index.append(idx)
    return(good_index)

In [599]:
only_year_good_index = get_good_index(only_year_df)

['1984']
['2000', '20']
['2001']
['1982']
['1998']
['2012']
['16', '1991']
['1988']
['25', '2014']
['2016']
['19', '1976']
['1981']
['2011']
['1997', '911', '911', '911', '300']
['2003']
['1983']
['1999']
['2010']
['1975']
['1972']
['2015']
['1989']
['14', '1994']
['1993']
['1996']
['2013']
['1974']
['1990']
['1995']
['2004']
['1987']
['1973']
['1992']
['1977']
['35', '1985']
['12', '969', '2007']
['2009']
['1986']
['1978']
['2002']
['1979']
['2006']
['2008']
['2005']
['1980']


In [602]:
combined_df[only_year_good_index] = only_year_df[only_year_good_index]

In [603]:
count_empties(combined_df)

Dates Missing 0
Dates Found 500


In [609]:
df.iloc[72]

'Lithium 0.25 (7/11/77).  LFTS wnl.  Urine tox neg.  Serum tox + fluoxetine 500; otherwise neg.  TSH 3.28.  BUN/Cr: 16/0.83.  Lipids unremarkable.  B12 363, Folate >20.  CBC: 4.9/36/308 Pertinent Medical Review of Systems Constitutional:\n'

### INDICES TO INVESTIGATE
- 72 (mm-dd-yyyy): two present
- 178, 198, 199 (Jan, Feb, ... format) : returns empty list
- 

In [630]:
"".join(list(combined_df[72][0]))

'7/11/77'

In [621]:
combined_df[72] = ['7/11/77']

In [816]:
print(combined_df)

0           [03/25/93]
1            [6/18/85]
2             [7/8/71]
3            [9/27/75]
4             [2/6/96]
5            [7/06/79]
6            [5/18/78]
7           [10/24/89]
8             [3/7/86]
9            [4/10/71]
10           [5/11/85]
11           [4/09/75]
12           [8/01/98]
13           [1/26/72]
14         [5/24/1990]
15         [1/25/2011]
16           [4/12/82]
17        [10/13/1976]
18           [4/24/98]
19           [5/21/77]
20           [7/21/98]
21          [10/21/79]
22           [3/03/90]
23           [2/11/76]
24        [07/25/1984]
25           [4-13-82]
26           [9/22/89]
27           [9/02/76]
28           [9/12/71]
29          [10/24/86]
            ...       
470             [1983]
471             [1999]
472             [2010]
473             [1975]
474             [1972]
475             [2015]
476             [1989]
477         [14, 1994]
478             [1993]
479             [1996]
480             [2013]
481             [1974]
482        

### Helper Functions

In [632]:
def count_empties(this_df):
    counter = 0
    for each in this_df:
        if (each == []) or (type(each) == str):
            counter+=1
    print('Dates Missing {}'.format(counter))
    found = this_df.shape[0] - counter
    print('Dates Found {}'.format(found))

In [637]:
def merge_found(x,y):
    if x == []:
        return y
    else:
        return x 

In [638]:
def not_null(a,b):
    if a.isnull():
        return b
    else:
        return a

In [639]:
def merge_missing(a,b):
    if type(a) == list:
        return a
    else:
        return b

In [647]:
def insert_missing(got_it,missing):
    if len(got_it) == 0:
        return missing
        return got_it

In [643]:
def get_good_index(temp_df):
    cnt = 0
    good_index = []
    for entry,idx in zip(temp_df,temp_df.index.tolist()):
        if type(entry) == list:
            if len(entry) > 0:
                print(entry)
                cnt += 1
                good_index.append(idx)
    return(good_index)

In [633]:
print(df.shape)
print(df.shape)

(500,)
(500,)


## Import the raw data

In [1221]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
test = df[:]
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

### Create Final DF for Sorting

In [1222]:
final_df = df[:]

### First Iteration: Find the Month / Day / Year formats

In [1223]:
# m(m)/d(d)/yy(yy), m(m)/d(d)/yy
# numbers only: 125
nums_only = r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}'
nums_only_df = final_df.str.findall(nums_only)

In [1224]:
nums_only_df[:50]

0       [03/25/93]
1        [6/18/85]
2         [7/8/71]
3        [9/27/75]
4         [2/6/96]
5        [7/06/79]
6        [5/18/78]
7       [10/24/89]
8         [3/7/86]
9        [4/10/71]
10       [5/11/85]
11       [4/09/75]
12       [8/01/98]
13       [1/26/72]
14     [5/24/1990]
15     [1/25/2011]
16       [4/12/82]
17    [10/13/1976]
18       [4/24/98]
19       [5/21/77]
20       [7/21/98]
21      [10/21/79]
22       [3/03/90]
23       [2/11/76]
24    [07/25/1984]
25       [4-13-82]
26       [9/22/89]
27       [9/02/76]
28       [9/12/71]
29      [10/24/86]
30    [03/31/1985]
31       [7/20/72]
32       [4/12/87]
33      [06/20/91]
34     [5/12/2012]
35       [3/15/83]
36       [2/14/73]
37       [5/24/88]
38     [7/27/1986]
39       [1-14-81]
40       [7-29-75]
41       [6/24/87]
42       [8/14/94]
43     [4/13/2002]
44       [8/16/82]
45     [2/15/1998]
46       [7/15/91]
47      [06/12/94]
48       [9/17/84]
49       [2/28/75]
dtype: object

In [1225]:
count_empties(nums_only_df)

Dates Missing 375
Dates Found 125


In [1226]:
# add nums_only dates to final_DF
nums_only_good_index = get_good_index(nums_only_df)
final_df[nums_only_good_index] = nums_only_df[nums_only_good_index]

['03/25/93']
['6/18/85']
['7/8/71']
['9/27/75']
['2/6/96']
['7/06/79']
['5/18/78']
['10/24/89']
['3/7/86']
['4/10/71']
['5/11/85']
['4/09/75']
['8/01/98']
['1/26/72']
['5/24/1990']
['1/25/2011']
['4/12/82']
['10/13/1976']
['4/24/98']
['5/21/77']
['7/21/98']
['10/21/79']
['3/03/90']
['2/11/76']
['07/25/1984']
['4-13-82']
['9/22/89']
['9/02/76']
['9/12/71']
['10/24/86']
['03/31/1985']
['7/20/72']
['4/12/87']
['06/20/91']
['5/12/2012']
['3/15/83']
['2/14/73']
['5/24/88']
['7/27/1986']
['1-14-81']
['7-29-75']
['6/24/87']
['8/14/94']
['4/13/2002']
['8/16/82']
['2/15/1998']
['7/15/91']
['06/12/94']
['9/17/84']
['2/28/75']
['11/22/75']
['5/24/91']
['6/13/92']
['7/11/71']
['12/26/86']
['10/11/1987']
['3/14/95']
['12/01/73']
['12/5/2010']
['08/20/1982']
['7/24/95']
['8/06/83']
['02/22/92']
['6/28/87']
['07/29/1994']
['08/11/78']
['10/29/91']
['7/6/91']
['1/21/87']
['11/3/1985']
['7/04/82']
['4-13-89']
['7/11/77', '9/36/308']
['4/12/74']
['09/19/81']
['9/6/79']
['12/5/87']
['01/05/1999']
['4/22/

In [1227]:
# check results
final_df

0                                             [03/25/93]
1                                              [6/18/85]
2                                               [7/8/71]
3                                              [9/27/75]
4                                               [2/6/96]
5                                              [7/06/79]
6                                              [5/18/78]
7                                             [10/24/89]
8                                               [3/7/86]
9                                              [4/10/71]
10                                             [5/11/85]
11                                             [4/09/75]
12                                             [8/01/98]
13                                             [1/26/72]
14                                           [5/24/1990]
15                                           [1/25/2011]
16                                             [4/12/82]
17                             

### Second Iteration: Find Alpha Numeric Dates

In [1228]:
# to grab dates with Month at the beginning
#alpha_present = r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{2}, )?\d{4}'
#alpha_present = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
alpha_v2 = r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|June|Jun|Jul|July|Aug|Sep|Sept|Oct|Nov|Dec)[a-z,.-]*[\s]*(?:\d{,2}[stndth,-]*[\s]*)?\d{4}'
alpha_present_df = final_df.str.findall(alpha_v2)
count_empties(alpha_present_df)

Dates Missing 157
Dates Found 343


In [1229]:
#add alphanumeric dates to final_DF
alpha_present_good_index = get_good_index(alpha_present_df)
final_df[alpha_present_good_index] = alpha_present_df[alpha_present_good_index]

['24 Jan 2001']
['10 Sep 2004']
['26 May 1982']
['28 June 2002']
['06 May 1972']
['25 Oct 1987']
['14 Oct 1996']
['30 Nov 2007']
['28 June 1994']
['14 Jan 1981']
['10 Oct 1985']
['11 February 1985']
['10 Feb 1983']
['05 Feb 1992']
['21 Oct 2012']
['14 Feb 1995']
['30 May 2016']
['22 Jan 1996']
['14 Oct 1992']
['06 Oct 2003']
['18 Oct 1999']
['11 Nov 2004']
['30 May 2001']
['02 Feb 1978']
['09 Sep 1989']
['12 March 1980']
['22 June 1990']
['28 Sep 2015']
['13 Jan 1972']
['06 Mar 1974']
['10 Oct 1974']
['26 May 1974']
['10 Feb 1990']
['23 Aug 2000']
['26 May 2001']
['21 Oct 2007']
['19 Oct 2016']
['05 Mar 1974']
['29 Jan 1994']
['21 Oct 1978']
['18 August 1975']
['11 Nov 1996']
['01 Oct 1979']
['13 Oct 1986']
['21 Oct 1995']
['24 Jan 2011']
['04 Oct 1972']
['23 Aug 1993']
['18 Oct 2006']
['04 Dec 1988']
['21 Oct 1983']
['26 May 2010']
['18 Jan 1990']
['15 Jun 1985']
['10 Dec 1982']
['09 Dec 1988']
['18 August 1995']
['13 June 1974']
['26 May 2008']
['11 Nov 2002']
['17 Aug 1985']
['13 Oc

### Find entries with Month/Year m(m)/yyyy format

In [1230]:
# regex for month/year
month_year = r'\d{,2}[/]+\d{2,4}'
month_year_df = final_df.str.findall(month_year)

In [1231]:
# indices of newly found entries
month_year_good_index = get_good_index(month_year_df)
final_df[month_year_good_index] = month_year_df[month_year_good_index]

['6/1998']
['6/2005']
['10/1973']
['9/2005']
['03/1980']
['12/2005']
['5/1987']
['5/2004']
['8/1974']
['3/1986']
['10/1997']
['3/1993']
['3/1981']
['9/2003']
['10/1993']
['1/1983']
['7/1994']
['12/2008']
['10/1980']
['8/2003']
['12/1975']
['11/2010']
['7/1997']
['7/2014']
['9/2001']
['8/1986']
['1/1978']
['9/1975']
['1/2009']
['5/1995']
['8/1989']
['11/2000']
['06/1973']
['2/1999']
['6/2001']
['12/1978']
['8/2009']
['7/1973']
['1/2014']
['09/1975']
['12/2012']
['5/1999']
['7/1989']
['7/2009']
['11/1998']
['3/1995']
['2/2009']
['08/1988']
['4/2007']
['5/2000']
['4/2012']
['10/2001']
['2/1977']
['8/2008']
['2/1983']
['5/1979']
['1/1992']
['11/2008']
['12/2014']
['4/1974']
['10/1981']
['10/1986']
['03/1973']
['12/1994']
['8/1999']
['10/2010']
['10/1994']
['01/2007']
['8/2010']
['03/1990']
['11/2016']
['4/2004']
['2/1973']
['7/1987']
['8/2000']
['8/1975']
['5/1977']
['3/2000']
['9/1984']
['4/1973']
['12/1986']
['4/1979']
['7/2004']
['11/1984']
['5/2016']
['12/2007']
['11/1982']
['06/1981']

In [1232]:
count_empties(final_df)

Dates Missing 45
Dates Found 455


### In Final DF, Fined Entires with Only Year Values

In [1233]:
# regex for year only
only_year = r'[a-zA-Z]?(\d{4})'
only_year_df = final_df.str.findall(only_year)

In [1234]:
# indices of newly found entries
only_year_good_index = get_good_index(only_year_df)

['1984']
['2000']
['2001']
['1982']
['1998']
['2012']
['1991']
['1988']
['2014']
['2016']
['1976']
['1981']
['2011']
['1997']
['2003']
['1983']
['1999']
['2010']
['1975']
['1972']
['2015']
['1989']
['1994']
['1993']
['1996']
['2013']
['1974']
['1990']
['1995']
['2004']
['1987']
['1973']
['1992']
['1977']
['1985']
['2007']
['2009']
['1986']
['1978']
['2002']
['1979']
['2006']
['2008']
['2005']
['1980']


In [1235]:
# Insert newly found entries for month/year into Combined DF, Final.
final_df[only_year_good_index] = only_year_df[only_year_good_index]

In [1236]:
count_empties(final_df)

Dates Missing 0
Dates Found 500


In [1237]:
final_df[455:]

455    [1984]
456    [2000]
457    [2001]
458    [1982]
459    [1998]
460    [2012]
461    [1991]
462    [1988]
463    [2014]
464    [2016]
465    [1976]
466    [1981]
467    [2011]
468    [1997]
469    [2003]
470    [1983]
471    [1999]
472    [2010]
473    [1975]
474    [1972]
475    [2015]
476    [1989]
477    [1994]
478    [1993]
479    [1996]
480    [2013]
481    [1974]
482    [1990]
483    [1995]
484    [2004]
485    [1987]
486    [1973]
487    [1992]
488    [1977]
489    [1985]
490    [2007]
491    [2009]
492    [1986]
493    [1978]
494    [2002]
495    [1979]
496    [2006]
497    [2008]
498    [2005]
499    [1980]
dtype: object

## data cleaning: fix errors and convert to standard format

In [1238]:
import dateutil

#### Fix Errors in Alpha_Present_DF

In [1248]:
idx = 228
for each in final_df[228:343]:
    temp = each[0].strip(" ")
    if "Janaury" in temp:
        temp = temp.replace("Janaury","Jan")
    elif "Decemeber" in temp:
        temp = temp.replace("Decemeber","Dec")
    temp2 = "1 "+ temp
    final_df.loc[idx] = [temp2]
#    print(temp2)
    idx+=1

In [1249]:
idx = 228
for each in final_df[228:343]:
    print(idx,each)
    idx+=1

228 ['1 September 1985']
229 ['1 June 2011']
230 ['1 May 1986']
231 ['1 May 2016']
232 ['1 July 1977']
233 ['1 July, 1990']
234 ['1 Jul 2003']
235 ['1 Oct 2015']
236 ['1 May 1995']
237 ['1 February 1976']
238 ['1 January 1995']
239 ['1 Feb 1978']
240 ['1 May, 2011']
241 ['1 May 2004']
242 ['1 Nov 2010']
243 ['1 Sep 2012']
244 ['1 January, 2013']
245 ['1 November 1990']
246 ['1 July 1981']
247 ['1 May 1983']
248 ['1 July 1995']
249 ['1 April 1993']
250 ['1 May 2005']
251 ['1 December 1998']
252 ['1 Jan 2007']
253 ['1 Feb 2016']
254 ['1 August 1979']
255 ['1 Oct 2014']
256 ['1 Aug 1988']
257 ['1 Sep 2015']
258 ['1 Apr 1976']
259 ['1 Nov 1979']
260 ['1 February 2000']
261 ['1 Oct 1986']
262 ['1 Jun 2002']
263 ['1 September 1981']
264 ['1 June 2007']
265 ['1 April 1989']
266 ['1 September 1999']
267 ['1 Oct 1980']
268 ['1 Dec 2009']
269 ['1 July 1992']
270 ['1 May, 2006']
271 ['1 August 2008']
272 ['1 Feb 1993']
273 ['1 February, 2010']
274 ['1 April 1985']
275 ['1 September 1984']
276 ['1

In [1251]:
# check results
for each in final_df.loc[340:345]:
    print(dateutil.parser.parse(each[0]))

1980-05-01 00:00:00
2007-11-01 00:00:00
1976-03-01 00:00:00
1998-06-10 00:00:00
2005-06-10 00:00:00
1973-10-10 00:00:00


#### Fix Errors in Month/Year

In [1252]:
for each in final_df[343:455]:
    temp = each[0].strip(" ")
    day_year = temp[:-4] + "1/" + temp[-4:]
    print(day_year)

6/1/1998
6/1/2005
10/1/1973
9/1/2005
03/1/1980
12/1/2005
5/1/1987
5/1/2004
8/1/1974
3/1/1986
10/1/1997
3/1/1993
3/1/1981
9/1/2003
10/1/1993
1/1/1983
7/1/1994
12/1/2008
10/1/1980
8/1/2003
12/1/1975
11/1/2010
7/1/1997
7/1/2014
9/1/2001
8/1/1986
1/1/1978
9/1/1975
1/1/2009
5/1/1995
8/1/1989
11/1/2000
06/1/1973
2/1/1999
6/1/2001
12/1/1978
8/1/2009
7/1/1973
1/1/2014
09/1/1975
12/1/2012
5/1/1999
7/1/1989
7/1/2009
11/1/1998
3/1/1995
2/1/2009
08/1/1988
4/1/2007
5/1/2000
4/1/2012
10/1/2001
2/1/1977
8/1/2008
2/1/1983
5/1/1979
1/1/1992
11/1/2008
12/1/2014
4/1/1974
10/1/1981
10/1/1986
03/1/1973
12/1/1994
8/1/1999
10/1/2010
10/1/1994
01/1/2007
8/1/2010
03/1/1990
11/1/2016
4/1/2004
2/1/1973
7/1/1987
8/1/2000
8/1/1975
5/1/1977
3/1/2000
9/1/1984
4/1/1973
12/1/1986
4/1/1979
7/1/2004
11/1/1984
5/1/2016
12/1/2007
11/1/1982
06/1/1981
4/1/2013
4/1/1999
5/1/2006
10/1/1978
12/1/1989
2/1/1974
11/1/1986
4/1/1983
10/1/2014
6/1/1989
9/1/1980
9/1/1992
9/1/2000
7/1/1981
1/1/2008
4/1/2002
7/1/1985
5/1/2010
8/1/2002


In [1253]:
# 343 - 455
idx = 343
for each in final_df[343:455]:
    temp = each[0].strip(" ")
    day_year = temp[:-4] + "1/" + temp[-4:]
    final_df.loc[idx] = [day_year]
    idx +=1

In [1255]:
# check results
for each in final_df.loc[[343,454]]:
    print(dateutil.parser.parse(each[0]))

1998-06-01 00:00:00
1982-07-01 00:00:00


### Add day month to the year only entries

In [1256]:
idx = 455
for each in final_df[455:]:
    temp = each[0].strip(" ")
    mo_day_year = "1/1/"+temp[-4:]
    final_df.loc[idx] = [mo_day_year]
#    print(mo_day_year)
    idx +=1

In [1260]:
date_time_equivalents = []
for idx,each in enumerate(final_df):
    temp = dateutil.parser.parse(each[0], dayfirst=False)
    date_time_equivalents.append(temp)


In [1261]:
clean_df = pd.Series(date_time_equivalents)
clean_df

0     1993-03-25
1     1985-06-18
2     1971-07-08
3     1975-09-27
4     1996-02-06
5     1979-07-06
6     1978-05-18
7     1989-10-24
8     1986-03-07
9     1971-04-10
10    1985-05-11
11    1975-04-09
12    1998-08-01
13    1972-01-26
14    1990-05-24
15    2011-01-25
16    1982-04-12
17    1976-10-13
18    1998-04-24
19    1977-05-21
20    1998-07-21
21    1979-10-21
22    1990-03-03
23    1976-02-11
24    1984-07-25
25    1982-04-13
26    1989-09-22
27    1976-09-02
28    1971-09-12
29    1986-10-24
         ...    
470   1983-01-01
471   1999-01-01
472   2010-01-01
473   1975-01-01
474   1972-01-01
475   2015-01-01
476   1989-01-01
477   1994-01-01
478   1993-01-01
479   1996-01-01
480   2013-01-01
481   1974-01-01
482   1990-01-01
483   1995-01-01
484   2004-01-01
485   1987-01-01
486   1973-01-01
487   1992-01-01
488   1977-01-01
489   1985-01-01
490   2007-01-01
491   2009-01-01
492   1986-01-01
493   1978-01-01
494   2002-01-01
495   1979-01-01
496   2006-01-01
497   2008-01-

In [1259]:
dateutil.parser.parse?

In [3]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here